# Using Recursive Feature Selection (RFS) to find: 
1. the most important feature to predict: (A) is_recid, (B) two_year_recid using logistic Regression
2. the combination of the most important features (2~3) to redict (A) is_recid, (B) two_year_recid


### TO DO: RFS with Neural Network 


"We compared the recidivism risk categories predicted by the COMPAS tool to the actual recidivism rates of defendants in the two years after they were scored, and found that the score correctly predicted an offender’s recidivism 61 percent of the time, but was only correct in its predictions of violent recidivism 20 percent of the time."

In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [2]:
compas = pd.read_csv('data/compas-scores-two-years-violent.csv')
compas.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid,two_year_recid.1
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1,1
2,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0,0
3,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,Low,2013-03-26,NaN,NaN,2,0,1102,0,0,0
4,7,marsha miles,marsha,miles,2013-11-30,Male,1971-08-22,44,25 - 45,Other,...,Low,2013-11-30,2013-11-30,2013-12-01,0,1,853,0,0,0


In [3]:
compas['c_jail_in'] = pd.to_datetime(compas['c_jail_in'], format='mixed')
compas['c_jail_out'] = pd.to_datetime(compas['c_jail_out'], format='mixed')

compas['v_screening_date'] = pd.to_datetime(compas['v_screening_date'], format='mixed')
compas['vr_offense_date'] = pd.to_datetime(compas['vr_offense_date'], format='mixed')

compas['c_offense_date'] = pd.to_datetime(compas['c_offense_date'], format='mixed')
compas['c_arrest_date'] = pd.to_datetime(compas['c_arrest_date'], format='mixed')

compas['compas_screening_date'] = pd.to_datetime(compas['compas_screening_date'], format='mixed')

In [4]:
compas = compas.drop(['first','last','c_case_number','c_charge_degree', 'c_charge_desc', 'r_case_number','r_charge_degree','vr_case_number','vr_charge_degree','start','end','event'],axis=1)

#remove row with nan scores 
compas = compas.loc[compas['score_text'].notnull()]
#clean out the "-1" decile score from dataset 
compas = compas.loc[compas['v_decile_score'] != -1]

#label encodimg
compas['sex'] = compas['sex'].replace({'Male': 1, 'Female': 0})
compas['score_text'] = compas['score_text'].replace({'Low':0, 'Medium':1, 'High': 2})

#days in jail
compas['jail_out- jail_in'] = (compas['c_jail_out'] - compas['c_jail_in']).dt.days

In [5]:
## Logistic Regression using two features (variables): age and prir count
'''
+--------------+--------------------+--------------------+--------------------+---------+
|    Metric    |     Precision      |       Recall       |      F1-Score      | Support |
+--------------+--------------------+--------------------+--------------------+---------+
|   Class 0    | 0.8513172140021653 | 0.8966172557962752 | 0.873380229544613  | 2631.0  |
|   Class 1    | 0.6954087346024636 | 0.601161665053243  | 0.6448598130841121 | 1033.0  |
|   accuracy   |                    |                    | 0.8133187772925764 | 3664.0  |
|  macro avg   | 0.7733629743023145 | 0.7488894604247591 | 0.7591200213143625 | 3664.0  |
| weighted avg | 0.8073615755687887 | 0.8133187772925764 | 0.8089529396418571 | 3664.0  |
+--------------+--------------------+--------------------+--------------------+---------+
'''

'\n+--------------+--------------------+--------------------+--------------------+---------+\n|    Metric    |     Precision      |       Recall       |      F1-Score      | Support |\n+--------------+--------------------+--------------------+--------------------+---------+\n|   Class 0    | 0.8513172140021653 | 0.8966172557962752 | 0.873380229544613  | 2631.0  |\n|   Class 1    | 0.6954087346024636 | 0.601161665053243  | 0.6448598130841121 | 1033.0  |\n|   accuracy   |                    |                    | 0.8133187772925764 | 3664.0  |\n|  macro avg   | 0.7733629743023145 | 0.7488894604247591 | 0.7591200213143625 | 3664.0  |\n| weighted avg | 0.8073615755687887 | 0.8133187772925764 | 0.8089529396418571 | 3664.0  |\n+--------------+--------------------+--------------------+--------------------+---------+\n'

In [6]:

compas['reoffend'] = compas['v_decile_score'].apply(lambda x: 1 if x > 5 else 0)

# Selecting features and target
features = compas[['age', 'priors_count']]
target = compas['reoffend']

# Splitting the data into training and testing sets
train_features, test_features, train_target, test_target = train_test_split(features, target, test_size=0.2, random_state=42)

# Initializing and training the logistic regression model
model = LogisticRegression()
model.fit(train_features, train_target)

# Making predictions
predicted_classes = model.predict(test_features)

# Evaluating the model
accuracy = accuracy_score(test_target, predicted_classes)
report = classification_report(test_target, predicted_classes)
conf_matrix = confusion_matrix(test_target, predicted_classes)

report

'              precision    recall  f1-score   support\n\n           0       0.87      0.95      0.91       765\n           1       0.68      0.42      0.52       183\n\n    accuracy                           0.85       948\n   macro avg       0.77      0.69      0.71       948\nweighted avg       0.83      0.85      0.83       948\n'

In [7]:
## features in consideration: 
'''
Data columns (total 43 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----                  
 3   sex                      4738 non-null   int64             
 5   age                      4738 non-null   int64         
 6   age_cat                  4738 non-null   object        
 7   race                     4738 non-null   object        
 8   juv_fel_count            4738 non-null   int64         
 9   decile_score             4738 non-null   int64         
 10  juv_misd_count           4738 non-null   int64         
 11  juv_other_count          4738 non-null   int64         
 12  priors_count             4738 non-null   int64             
 18  c_days_from_compas       4727 non-null   float64       
 19  is_recid                 4738 non-null   int64                    
 25  violent_recid            0 non-null      float64       
 26  is_violent_recid         4738 non-null   int64           
 30  decile_score.1           4738 non-null   int64         
 31  score_text               4738 non-null   int64              
 34  v_decile_score           4738 non-null   int64         
 35  v_score_text             4738 non-null   object        
 39  priors_count.1           4738 non-null   int64         
 40  two_year_recid           4738 non-null   int64         
 41  two_year_recid.1         4738 non-null   int64         
 42  jail_out- jail_in        4470 non-null   float64     
'''

'\nData columns (total 43 columns):\n #   Column                   Non-Null Count  Dtype         \n---  ------                   --------------  -----                  \n 3   sex                      4738 non-null   int64             \n 5   age                      4738 non-null   int64         \n 6   age_cat                  4738 non-null   object        \n 7   race                     4738 non-null   object        \n 8   juv_fel_count            4738 non-null   int64         \n 9   decile_score             4738 non-null   int64         \n 10  juv_misd_count           4738 non-null   int64         \n 11  juv_other_count          4738 non-null   int64         \n 12  priors_count             4738 non-null   int64             \n 18  c_days_from_compas       4727 non-null   float64       \n 19  is_recid                 4738 non-null   int64                    \n 25  violent_recid            0 non-null      float64       \n 26  is_violent_recid         4738 non-null   int64           \n 30

In [8]:
from sklearn.feature_selection import RFE
from sklearn.impute import SimpleImputer

In [20]:

feature_list = ['sex','age','age_cat','race','juv_fel_count','juv_misd_count',
                      'juv_other_count','priors_count','c_days_from_compas' ,
                      'priors_count.1', 'jail_out- jail_in']
all_feature = compas[feature_list]
#clean and label features 
#all_feature.loc[:,'v_score_text'] = all_feature['v_score_text'].replace({'Low':0, 'Medium':1, 'High': 2})
all_feature.loc[:, 'race'] = LabelEncoder().fit_transform(all_feature['race'])
all_feature.loc[:,'age_cat'] = all_feature['age_cat'].replace({'Greater than 45': 2, '25 - 45': 1, 'Less than 25': 0})

In [27]:
#feature = all_features
target = compas['two_year_recid']

# Handling missing values using SimpleImputer
imputer = SimpleImputer(strategy='mean')  # You can choose a different strategy based on your data
features_imputed = imputer.fit_transform(all_feature)


#split data 
train_features, test_features, train_target, test_target = train_test_split(features_imputed, target, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=1000)

# Using RFE to select features
rfe = RFE(model, n_features_to_select=2)  # You can adjust the number of features to select
fit = rfe.fit(train_features, train_target)

# Printing the selected features
selected_features = pd.DataFrame({'Feature': feature_list, 'Ranking': fit.ranking_})
selected_features = selected_features.sort_values(by='Ranking')
print(selected_features)

# Selecting only the important features
selected_features = selected_features[selected_features['Ranking'] == 1]['Feature']
selected_features = list(selected_features)

# Training the logistic regression model with selected features
model.fit(train_features[:, fit.support_], train_target)

# Making predictions
predicted_classes = model.predict(test_features[:, fit.support_])


# Evaluating the model
accuracy = accuracy_score(test_target, predicted_classes)
report = classification_report(test_target, predicted_classes)
conf_matrix = confusion_matrix(test_target, predicted_classes)

print(f"Selected Features: {selected_features}")
print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)
print("Confusion Matrix:\n", conf_matrix)

               Feature  Ranking
0                  sex        1
2              age_cat        1
5       juv_misd_count        2
9       priors_count.1        3
6      juv_other_count        4
3                 race        5
4        juv_fel_count        6
7         priors_count        7
1                  age        8
10   jail_out- jail_in        9
8   c_days_from_compas       10
Selected Features: ['sex', 'age_cat']
Accuracy: 0.8343881856540084
Classification Report:
               precision    recall  f1-score   support

           0       0.83      1.00      0.91       791
           1       0.00      0.00      0.00       157

    accuracy                           0.83       948
   macro avg       0.42      0.50      0.45       948
weighted avg       0.70      0.83      0.76       948

Confusion Matrix:
 [[791   0]
 [157   0]]


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## RFE with Target as is_recid a

In [13]:

feature_list = ['sex','age','age_cat','race','juv_fel_count','juv_misd_count',
                      'juv_other_count','priors_count','c_days_from_compas',
                      'priors_count.1', 'jail_out- jail_in']

all_feature = compas[feature_list]
#clean and label features 
all_feature.loc[:, 'race'] = LabelEncoder().fit_transform(all_feature['race'])
all_feature.loc[:,'age_cat'] = all_feature['age_cat'].replace({'Greater than 45': 2, '25 - 45': 1, 'Less than 25': 0})

In [31]:
#feature = all_features
target = compas['is_recid']

# Handling missing values using SimpleImputer
imputer = SimpleImputer(strategy='mean') 
features_imputed = imputer.fit_transform(all_feature)


#split data 
train_features, test_features, train_target, test_target = train_test_split(features_imputed, target, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=1000)

# Using RFE to select features
rfe = RFE(model, n_features_to_select=3) # change this
fit = rfe.fit(train_features, train_target)

# Printing features rank 
selected_features = pd.DataFrame({'Feature': feature_list, 'Ranking': fit.ranking_})
selected_features = selected_features.sort_values(by='Ranking')
print(selected_features)

# select important features
selected_features = selected_features[selected_features['Ranking'] == 1]['Feature']
selected_features = list(selected_features)

# Training the logistic regression model with selected features
model.fit(train_features[:, fit.support_], train_target)

# get coeff
feature_weights = pd.DataFrame({'Feature': selected_features, 'Weight': model.coef_[0]})
feature_weights = feature_weights.sort_values(by='Weight', ascending=False)

# feature weights
print("Feature Weights:")
print(feature_weights)
print()

predicted_classes = model.predict(test_features[:, fit.support_])

accuracy = accuracy_score(test_target, predicted_classes)
report = classification_report(test_target, predicted_classes, zero_division=1)
conf_matrix = confusion_matrix(test_target, predicted_classes)

print(f"Selected Features: {selected_features}")
print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)
print("Confusion Matrix:\n", conf_matrix)

               Feature  Ranking
0                  sex        1
2              age_cat        1
5       juv_misd_count        1
4        juv_fel_count        2
9       priors_count.1        3
6      juv_other_count        4
3                 race        5
7         priors_count        6
1                  age        7
10   jail_out- jail_in        8
8   c_days_from_compas        9
Feature Weights:
          Feature    Weight
0             sex  0.750811
2  juv_misd_count  0.571391
1         age_cat -0.533797

Selected Features: ['sex', 'age_cat', 'juv_misd_count']
Accuracy: 0.79957805907173
Classification Report:
               precision    recall  f1-score   support

           0       0.80      1.00      0.89       753
           1       1.00      0.03      0.05       195

    accuracy                           0.80       948
   macro avg       0.90      0.51      0.47       948
weighted avg       0.84      0.80      0.72       948

Confusion Matrix:
 [[753   0]
 [190   5]]


In [ ]:
'''
Violent Feature  Ranking
0                  sex        1
2              age_cat        1
5       juv_misd_count        1
4        juv_fel_count        2
9       priors_count.1        3
6      juv_other_count        4
3                 race        5
7         priors_count        6
1                  age        7
10   jail_out- jail_in        8
8   c_days_from_compas        9
Feature Weights:
          Feature    Weight
0             sex  0.717787
2  juv_misd_count  0.563509
1         age_cat -0.581757

Selected Features: ['sex', 'age_cat', 'juv_misd_count']
Accuracy: 0.8333333333333334
Classification Report:
               precision    recall  f1-score   support

           0       0.83      1.00      0.91       785
           1       1.00      0.03      0.06       163
'''

In [ ]:
'''
              Feature  Ranking
0                  sex        1
2              age_cat        1
5       juv_misd_count        1
4        juv_fel_count        2
9       priors_count.1        3
6      juv_other_count        4
3                 race        5
7         priors_count        6
1                  age        7
10   jail_out- jail_in        8
8   c_days_from_compas        9
Feature Weights:
          Feature    Weight
0             sex  0.750811
2  juv_misd_count  0.571391
1         age_cat -0.533797

Selected Features: ['sex', 'age_cat', 'juv_misd_count']
Accuracy: 0.79957805907173
Classification Report:
               precision    recall  f1-score   support

           0       0.80      1.00      0.89       753
           1       1.00      0.03      0.05       195
...

Confusion Matrix:
 [[753   0]
 [190   5]]
'''